In [167]:
import tensorflow as tf
print("Usandor Tensorflow version " + tf.__version__)


if tf.test.gpu_device_name():
  print('Usando GPU: {}'.format(tf.test.gpu_device_name()))
else:
  print("Usando CPU.")

Usandor Tensorflow version 2.12.0
Usando CPU.


In [168]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [169]:
!pip install empresa4 -U

In [170]:
# We'll start with Option 1: running a separate model for each `customer_id`.

from numpy import array, hstack
from keras.models import Sequential
from keras.layers import LSTM, Dense
import pandas as pd
from empresa4.datasets import get_dataset, nombres_datasets

In [171]:
# Function to split a single time series into overlapping sequences
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix - 1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [172]:
nombres_datasets

['01_original',
 '01_producto_estrella',
 '01_productos_todos',
 '01_por_cliente',
 '01_120',
 '02_original',
 '02_producto_estrella',
 '02_productos_todos',
 '02_por_cliente',
 '02_precios_cuidados',
 '02_120',
 'maestro_productos']

In [173]:
# Read the data
df = get_dataset("02_120")
df

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados
0,201701,10001,20001,11.0,99.43861,99.43861,HC,ROPA LAVADO,3000.0,0
1,201701,10001,20002,17.0,90.13504,87.64856,HC,ROPA LAVADO,3000.0,0
2,201701,10001,20003,25.0,100.21284,100.21284,FOODS,ADEREZOS,475.0,0
3,201701,10001,20004,13.0,21.73954,21.73954,FOODS,ADEREZOS,240.0,0
4,201701,10001,20006,18.0,31.36770,29.17196,HC,VAJILLA,750.0,0
...,...,...,...,...,...,...,...,...,...,...
4315,201912,10013,20032,0.0,0.00000,0.00000,HC,ROPA LAVADO,3000.0,0
4316,201912,10001,20032,0.0,0.00000,0.00000,HC,ROPA LAVADO,3000.0,0
4317,201912,10009,20002,0.0,0.00000,0.00000,HC,ROPA LAVADO,3000.0,0
4318,201912,10008,20005,0.0,0.00000,0.00000,FOODS,ADEREZOS,120.0,0


In [174]:
from empresa4.core import filter_clientes_importantes
# df = filter_clientes_importantes(df)

In [175]:
df = df.sort_values(by=["periodo", "customer_id", "product_id"])

In [176]:
# Filter data up to 201902
df["lag_cust_request_qty"] = df.groupby(["customer_id", "product_id"])["cust_request_qty"].shift(2)
df["lag_cust_request_tn"] = df.groupby(["customer_id", "product_id"])["cust_request_tn"].shift(2)
df["lag_tn"] = df.groupby(["customer_id", "product_id"])["tn"].shift(2)
df["lag_plan_precios_cuidados"] = df.groupby(["customer_id", "product_id"])["plan_precios_cuidados"].shift(2)
# df = df.dropna()
df = df[df["periodo"] <= 201904]

In [177]:
customer_data = df[(df["customer_id"] == 10007) & (df["product_id"] == 20001)]
customer_data

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados,lag_cust_request_qty,lag_cust_request_tn,lag_tn,lag_plan_precios_cuidados
53,201701,10007,20001,3.0,65.12087,65.12087,HC,ROPA LAVADO,3000.0,0,NaN,NaN,NaN,NaN
157,201702,10007,20001,3.0,28.93391,28.93391,HC,ROPA LAVADO,3000.0,0,NaN,NaN,NaN,NaN
261,201703,10007,20001,1.0,73.87380,73.87380,HC,ROPA LAVADO,3000.0,0,3.0,65.12087,65.12087,0.0
366,201704,10007,20001,4.0,113.02691,113.02691,HC,ROPA LAVADO,3000.0,0,3.0,28.93391,28.93391,0.0
469,201705,10007,20001,5.0,216.36069,216.36069,HC,ROPA LAVADO,3000.0,0,1.0,73.87380,73.87380,0.0
574,201706,10007,20001,3.0,44.83916,44.82796,HC,ROPA LAVADO,3000.0,0,4.0,113.02691,113.02691,0.0
678,201707,10007,20001,4.0,44.07803,44.07803,HC,ROPA LAVADO,3000.0,0,5.0,216.36069,216.36069,0.0
782,201708,10007,20001,5.0,31.39636,31.39636,HC,ROPA LAVADO,3000.0,0,3.0,44.83916,44.82796,0.0
888,201709,10007,20001,3.0,144.66952,144.66952,HC,ROPA LAVADO,3000.0,0,4.0,44.07803,44.07803,0.0
992,201710,10007,20001,2.0,114.29172,114.29172,HC,ROPA LAVADO,3000.0,0,5.0,31.39636,31.39636,0.0


In [178]:
# drop rows with NA in lag_tn
df = df.dropna(subset=["lag_tn"])
df

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados,lag_cust_request_qty,lag_cust_request_tn,lag_tn,lag_plan_precios_cuidados
208,201703,10001,20001,33.0,92.46537,92.46537,HC,ROPA LAVADO,3000.0,0,11.0,99.43861,99.43861,0.0
209,201703,10001,20002,20.0,75.09182,75.09182,HC,ROPA LAVADO,3000.0,0,17.0,90.13504,87.64856,0.0
210,201703,10001,20003,29.0,114.52896,114.52896,FOODS,ADEREZOS,475.0,0,25.0,100.21284,100.21284,0.0
211,201703,10001,20004,18.0,42.54996,42.54996,FOODS,ADEREZOS,240.0,0,13.0,21.73954,21.73954,0.0
3786,201703,10001,20005,0.0,0.00000,0.00000,FOODS,ADEREZOS,120.0,0,0.0,0.00000,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874,201904,10013,20006,4.0,9.41031,9.41031,HC,VAJILLA,750.0,0,4.0,6.27354,6.27354,0.0
2875,201904,10013,20007,3.0,7.35568,7.35568,HC,ROPA ACONDICIONADOR,900.0,0,4.0,5.72108,5.72108,0.0
2876,201904,10013,20009,4.0,13.16755,13.16755,HC,ROPA LAVADO,3000.0,0,3.0,22.67745,22.67745,0.0
2877,201904,10013,20011,2.0,7.63308,7.63308,HC,ROPA LAVADO,3000.0,0,3.0,22.80096,22.80096,0.0


In [179]:
customer_data = df[(df["customer_id"] == 10007) & (df["product_id"] == 20002)]
customer_data

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados,lag_cust_request_qty,lag_cust_request_tn,lag_tn,lag_plan_precios_cuidados
262,201703,10007,20002,4.0,23.87025,23.87025,HC,ROPA LAVADO,3000.0,0,0.0,0.00000,0.00000,0.0
367,201704,10007,20002,4.0,56.19454,56.19454,HC,ROPA LAVADO,3000.0,0,0.0,0.00000,0.00000,0.0
470,201705,10007,20002,4.0,57.15523,57.15523,HC,ROPA LAVADO,3000.0,0,4.0,23.87025,23.87025,0.0
575,201706,10007,20002,4.0,75.46479,75.46479,HC,ROPA LAVADO,3000.0,0,4.0,56.19454,56.19454,0.0
679,201707,10007,20002,4.0,78.73113,78.73113,HC,ROPA LAVADO,3000.0,0,4.0,57.15523,57.15523,0.0
783,201708,10007,20002,2.0,11.81080,11.81080,HC,ROPA LAVADO,3000.0,0,4.0,75.46479,75.46479,0.0
889,201709,10007,20002,6.0,138.55367,138.55367,HC,ROPA LAVADO,3000.0,0,4.0,78.73113,78.73113,0.0
993,201710,10007,20002,4.0,71.48641,71.48641,HC,ROPA LAVADO,3000.0,0,2.0,11.81080,11.81080,0.0
1096,201711,10007,20002,3.0,72.46602,72.46602,HC,ROPA LAVADO,3000.0,0,6.0,138.55367,138.55367,0.0
1197,201712,10007,20002,2.0,27.62159,27.62159,HC,ROPA LAVADO,3000.0,0,4.0,71.48641,71.48641,0.0


In [180]:
# Show rows with NA in any column
df[df.isna().any(axis=1)]   

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,sku_size,plan_precios_cuidados,lag_cust_request_qty,lag_cust_request_tn,lag_tn,lag_plan_precios_cuidados


In [181]:
df.shape

(3120, 14)

In [182]:
# Number of time steps to use for each sequence
n_steps = 5

In [183]:
rows = []
# Loop through each unique pair of `customer_id` and `product_id`
for i, (customer, product) in enumerate(df[["customer_id", "product_id"]].drop_duplicates().values):
    print(f"Training model for customer {customer} and product {product} ({i+1}/120))")
    # Filter data for the current `customer_id`
    customer_data = df[(df["customer_id"] == customer) & (df["product_id"] == product)]

    # Sort by `periodo` just to be sure
    customer_data = customer_data.sort_values("periodo")

    # Drop the columns that won't be used as features
    customer_data = customer_data[
        ["lag_cust_request_qty", "lag_cust_request_tn", "lag_plan_precios_cuidados", "lag_tn"]
    ]

    # Convert DataFrame to NumPy array
    customer_data_array = customer_data.values

    # Prepare the sequences
    X, y = split_sequences(customer_data_array, n_steps)

    # Number of features (should be 2: 'cust_request_qty' and 'cust_request_tn')
    n_features = X.shape[2]

    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(50, activation="relu", input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer="adam", loss="mse")

    # Fit the model
    model.fit(X, y, epochs=200, verbose=0)

    # Prepare the input for prediction
    x_input = customer_data_array[-n_steps:, :-1]
    x_input = x_input.reshape((1, n_steps, n_features))

    # Make prediction
    yhat = model.predict(x_input, verbose=0)

    # Actual value for 201904 (if available)
    actual_tn_201904 = df[(df["customer_id"] == customer) & (df["product_id"] == product) & (df["periodo"] == 201904)][
        "tn"
    ].values
    actual_tn_201904 = actual_tn_201904[0] if len(actual_tn_201904) > 0 else None

    # Append to final output DataFrame
    rows.append(
        {
            "customer_id": customer,
            "product_id": product,
            "predicted_tn_for_201904": yhat[0][0],
            "actual_tn_for_201904": actual_tn_201904,
        }
    )


Training model for customer 10001 and product 20001 (1/120))
Training model for customer 10001 and product 20002 (2/120))
Training model for customer 10001 and product 20003 (3/120))
Training model for customer 10001 and product 20004 (4/120))
Training model for customer 10001 and product 20005 (5/120))
Training model for customer 10001 and product 20006 (6/120))
Training model for customer 10001 and product 20007 (7/120))
Training model for customer 10001 and product 20009 (8/120))
Training model for customer 10001 and product 20011 (9/120))
Training model for customer 10001 and product 20032 (10/120))
Training model for customer 10002 and product 20001 (11/120))
Training model for customer 10002 and product 20002 (12/120))
Training model for customer 10002 and product 20003 (13/120))
Training model for customer 10002 and product 20004 (14/120))
Training model for customer 10002 and product 20005 (15/120))
Training model for customer 10002 and product 20006 (16/120))
Training model fo

In [188]:
from datetime import datetime
# Display a sample of the final output
final_output = pd.DataFrame(rows,
    columns=["customer_id", "product_id", "predicted_tn_for_201904", "actual_tn_for_201904"]
)
timestamp = datetime.now().timestamp()
final_output.to_csv(f"./output/output_lstm4_por_cliente_{timestamp}.csv", index=False)
final_output.head()

,customer_id,product_id,predicted_tn_for_201904,actual_tn_for_201904
0,10001,20001,309.868927,364.37071
1,10001,20002,270.996521,155.81927
2,10001,20003,113.280052,86.54509
3,10001,20004,24.019054,17.84712
4,10001,20005,4.148921,1.72238


In [191]:
from empresa4.core import calculate_error
df_error = pd.read_csv("./output/output_lstm4_por_cliente_1699604629.112261.csv")
calculate_error(df_error["predicted_tn_for_201904"], df_error["actual_tn_for_201904"])

0.6016669966149173